                                           Image Processing 5 Hewan Mamalia

                                                       Kelompok 1 
                                                      Kelas A1 2021
##### Anggota Kelompok :

##### Bayu Abdurosyid - 2109106003
##### Alif Maulana Setyawan  - 2109106002
##### Ananta Yusra Putra.A - 2109106024

link Dataset : <a href="https://www.kaggle.com/datasets/shiv28/animal-5-mammal">Animal -5 Mammal</a>

- Install package dibawah ini, yang akan digunakan untuk melakukan split folder yang akan digunakan untuk folder train dan validation

In [ ]:
!pip install split_folders 

#### Library Utama

In [ ]:
import tensorflow as tf

In [ ]:
# Panggil Library untuk split
import splitfolders

#### Membuat Direktori Untuk Membaca Dataset

In [ ]:
dir = 'D:/UNMUL/vscode/PAKBBB/dataset'

#### Spliting Folder 80% training 20% validasi

- Hanya di gunakan sekali, jika sudah dilakukan spliting folder maka code ini tidak perlu di jalankan lagi.

In [ ]:
splitfolders.ratio(dir, output = 'D:/UNMUL/vscode/PAKBBB', ratio = (.8, .2))

#### Membuat Variabel Untuk Menentukan Resolusi Gambar

In [ ]:
# Resolusi/dimensi Width (w), high (h)
w = 150
h = 150

#### Library OS

In [ ]:
import os

Digunakan untuk :

- scandir = Mengakses semua file dan folder yang ada di lokasi yang ditentukan
- is_file = Untuk mengetahui apakah jenis objek tersebut berupa file atau direktori.

### Diagram Bar

- Digunakan Untuk Melihat Jumlah Data Image

In [ ]:
elephant = 0
horse = 0
lion = 0
cat = 0
dog = 0

array =  [ 
          'D:/UNMUL/vscode/PAKBBB/dataset/ELEPHANT', 
          'D:/UNMUL/vscode/PAKBBB/dataset/HORSE', 
          'D:/UNMUL/vscode/PAKBBB/dataset/LION', 
          'D:/UNMUL/vscode/PAKBBB/dataset/cat', 
          'D:/UNMUL/vscode/PAKBBB/dataset/dog'
          ]

for path in os.scandir(array[0]):
  if path.is_file():
    elephant += 1
for path in os.scandir(array[1]):
  if path.is_file():
    horse += 1
for path in os.scandir(array[2]):
  if path.is_file():
    lion += 1
for path in os.scandir(array[3]):
  if path.is_file():
    cat += 1
for path in os.scandir(array[4]):
  if path.is_file():
    dog += 1

print('file count:', elephant)
print('file count:', horse)
print('file count:', lion)
print('file count:', cat)
print('file count:', dog)

animal = ['elephant', 'horse', 'lion', 'cat', 'dog']
count = [elephant, horse, lion, cat, dog]

#### Import Matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = animal
y = count

plt.figure(figsize = (10,5))
plt.bar(x, y, color=['gray', 'brown', 'orange', 'yellow', 'black'])
plt.title("Jumlah Gambar Animal")
plt.show()

#### Membuat Class Name

In [ ]:
class_names = ['elephant', 'horse', 'lion', 'cat', 'dog']

### Analisis Sebelum Augmentasi

- Preprocessing

datagenbefore digunakan sebagai metode awal augmentasi yang akan digunakan dalam train_before <br>
dan train_before digunakan sebagai augmentasi yang akan digunakan pada data

- rescale = mengubah ukuran gambar
- class_mode = metode pemilihan klasifikasi
- subset = menentukan penggunaan data
- target_size = dimensi dari image yang akan digunakan dalam proses
- batch_size = banyaknya image yang akan dimasukan dalam setiap step

In [ ]:
datagenbefore = ImageDataGenerator(
    rescale = 1./255
)

train_before = datagenbefore.flow_from_directory(
    'D:/UNMUL/vscode/PAKBBB/train',
    class_mode = 'categorical',
    subset = 'training',
    target_size = (w, h),
    batch_size = 256
)

#### Visualisasi Sebelum Augmentasi

In [ ]:
plt.figure(figsize=(10, 10))

images, labels = train_before.next()

for i in range (16):
  plt.subplot(4, 4, i+1)
  plt.imshow(train_before[0][0][i])
  plt.title(class_names[tf.argmax(labels[i])])
  plt.axis('off')

# Augmentasi Gambar

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode = 'nearest',
    horizontal_flip = True,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.15,
)


In [ ]:
w = 150
h = 150

train_dataset = datagen.flow_from_directory(
    '/content/dataset/train',
    class_mode='categorical',
    target_size = (w, h),
    batch_size = 256
)

validation_dataset = datagen.flow_from_directory(
    '/content/dataset/val',
    class_mode='categorical',
    target_size = (w, h),
    batch_size = 256
)


In [ ]:
plt.figure(figsize=(10, 10))

images, labels = train_dataset.next()

for i in range (16):
  plt.subplot(4, 4, i+1)
  plt.imshow(train_dataset[0][0][i])
  plt.title(class_names[tf.argmax(labels[i])])
  plt.axis('off')

In [ ]:
model = Sequential(
    [
    layers.Conv2D(64, (3,3), activation='relu', input_shape=(w, h, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.5),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(512, activation = 'relu'),
    layers.Dense(256, activation = 'relu'),
    layers.Dense(5, activation = 'softmax')
    ]
)

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)
model.summary()

In [ ]:
class callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epocj, logs = None):
    if (logs.get('accuracy') > 0.95 and logs.get('val_accuracy') > 0.95):
      print('Akurasi telah mencapai 95%!')
      self.model.stop_training = True

In [ ]:
history = model.fit(
    train_dataset,
    validation_data = validation_dataset,
    validation_steps = 5,
    epochs = 17,
    callbacks = [callback()]    
)

In [ ]:
model.save('gfgModel.h5')
print('Model Saved!')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
model.evaluate(validation_dataset)